# UBS Challenge Preliminary Attempts

------

### Download necessary packages & Add system executable

In [ ]:
!pip install numpy pandas matplotlib --quiet
import numpy as np, pandas as pd, matplotlib
import os, sys; sys.path.insert(0, os.path.join(sys.path[0], '..'))

#### Download the files

In [ ]:
!pip install requests --quiet
import os, sys; sys.path.insert(0, os.path.join(sys.path[0], '..'))
from scripts.download_files import download_files
download_files()

## Project Motivation

### The Challenge

- The ```structure of the product is relatively complex```, and its ```interval accumulation characteristics are relatively idiosyncratic``` and there is ```no simple numerical analytical solution```. Such products often use numerical simulation (Monte Carlo, etc.) methods to price and calculate related risk exposures.

- The calculation results of the Monte Carlo model are ```more accurate```, but there are problems such as ```slow calculation speed and low sensitivity to changes in the external environment```. 

### The Task

基于所提供的 ```模拟产品特征、历史风险因子的数据``` 以及 ```市场环境```，建立能 ```准确、高效预测该类产品风险敞口``` 的模型 （例如AI），检验 ```模型的效率和准确度```。

Based on the provided ```simulated product characteristics, historical risk factor data and market environment```, establish a model (such as AI) that can ```accurately and efficiently predict the risk exposure``` of this type of product, and test ```the efficiency and accuracy``` of the model.

## Understand the Question

### What are the product characteristics from the provided .csv files? 

The product characteristics can be obtained from the ```trade-information.csv``` file. This file provides information about the trades: 

- ```Trade Name``` (identifier)

- ```Underlying```

- ```Pay Frequency```

- ```Lower Bound```

- ```Upper Bound```

### What are the product historical risk factors from the provided .csv files? 

The historical risk factors can be found in the ```trade-price-ir-vegas.csv``` file. 

This file contains the timeseries data for each trade, indicating the risks: 

- ```TV``` (Total Value)

- ```Vega```

Both historical risk factors are correlated with the historical trade data in the same ```trade-price-ir-vegas.csv``` file: 

- ```Zero Rate Shock```

- ```Expiry Bucket```

- ```Tenor Bucket```

### What are the market environment factors from the provided .csv files? 

The market environment factors can be found in the ```market-data-swap-rates.csv``` and ```market-data-swaption-vols.csv``` files. These files provide timeseries data for ```Swap Rate``` and ```Vols``` (implied normal volatilities). 

Regarding the factors influencing the ```Swap Rate```: 

- ```Start Date``` from ```market-data-swap-rates.csv```. The start date itself may not be too helpful, but the difference between the ```Start Date``` and ```Date``` may be useful. 

- ```Tenor``` (i.e., maturity time) from ```market-data-swap-rates.csv```

Regarding the factors including the ```Vols```: 

- ```Expiry``` from ```market-data-swaption-vols.csv```

- ```Tenor``` from ```market-data-swaption-vols.csv```

- ```Strike``` from ```market-data-swaption-vols.csv```

In the end, both ```Swap Rate``` and ```Vols``` are market indicators that may influence investors' decisions. In the preliminary stage, it is suggested to only take ```Swap Rate``` and ```Vols``` to represent market influences. 

### What risk exposure indicators should the model predict? 

The model should predict the ```TV``` (Total Value) and ```Vega``` of the trades. These are the main risk exposure indicators provided in the ```trade-price-ir-vegas.csv``` file. 

Summarized from the above analysis, after aggregating the data into specific day intervals (min. daily style), the correlation between ```TV``` and ```Vega``` and each potential aspect can be analyzed by machine learning methods, e.g., regression, Monte Carlo methods, and more. There are three aspects: 

- Aspect 1 (from the product characteristic perspective): 

  - Input (Independent Variables): ```Underlying```, ```Pay Frequency```, ```Lower Bound``` and ```Upper Bound``` \[AGGREGATED\] (in ```trade-information.csv```)

  - Predict (Dependent Variables): ```TV```, ```Vega```

- Aspect 2 (from the historical risk exposure indicator perspective): 

  - Input (Independent Variables): ```Zero Rate Shock```, ```Expiry Bucket```, and ```Tenor Bucket``` \[AGGREGATED\] (in ```trade-price-ir-vegas.csv```)

  - Predict (Dependent Variables): ```TV```, ```Vega```

- Aspect 3 (from the market environment factor perspective): 

  - Input (Independent Variables): ```Swap Rate``` \[AGGREGATED\] (in ```market-data-swap-rates.csv```), and ```Vols``` \[AGGREGATED\] (in ```market-data-swaption-vols.csv```) + ```Start Date``` and ```Tenor``` \[AGGREGATED\] (in ```market-data-swap-rates.csv```), ```Expiry```, ```Tenor``` and ```Strike``` (in ```market-data-swaption-vols.csv```)

  - Predict (Dependent Variables): ```TV```, ```Vega```

### How to evaluate the model efficiency and accuracy? 

Use standard machine learning evaluation metrics: 

- ```Mean Squared Error``` (MSE)

- ```R-squared```

- ```Other relevant metrics``` depending on the model.

### How the provided data can be deployed in machine prediction with Monte Carlo methods? 

The provided data can be used to train a machine learning model, such as a regression model to predict the TV (Total Value) and Vega of the trades. Here is the execution plan: 

- Attempt 1: ```Regression```

- Attempt 2: ```Monte Carlo Methods``` (previous work applicable)

- Attempt 3: ```Deep Learning Methods``` (previous work applicable)

## Data Processing

### Data Aggregation

**Data Aggregation for one column**

As what you may have noticed, the file ```trade-price-ir-vegas``` is extremely large, containing millions of lines. 
A smaller file, ```trade-price-ir-vegas-20220902``` is created in the ```assets``` folder. ChatGPT was invited to write a function to aggregate all data on top of a specific column besides the ```Value Date```, the ```TV```, and the ```Vega```. 

**Data Aggregation for many columns**

Next, ChatGPT was invited to modify the function definition so that it can only consider specific values ​​of the input column (optional) when aggregating multiple rows at the same time.

The script returned by ChatGPT is modified and added to ```scripts/aggregate_data.py```

#### Aggregate TV for one day

Consider Aspect 2: Historical Risk Exposure

- Group by ```Zero Rate Shock```

- Group by ```Expiry Bucket```

- Group by ```Tenor Bucket```

In [ ]:
import os, sys; sys.path.insert(0, os.path.join(sys.path[0], '..'))
from scripts.data_processing_visualization import aggregate
aspect_2_ZRS_TV_20220902 = aggregate(os.path.join("assets", "trade-price-ir-vegas-20220902.csv"), 
                        {
                                "Value Date": {}, 
                                "Zero Rate Shock": {}, 
                                "Expiry Bucket": {"aggregate": True}, 
                                "Tenor Bucket": {"aggregate": True}, 
                        }, 
                        {
                                "TV": {}
                        })

from IPython.display import display
display(aspect_2_ZRS_TV_20220902)

In [ ]:
import os, sys; sys.path.insert(0, os.path.join(sys.path[0], '..'))
from scripts.data_processing_visualization import aggregate
aspect_2_EB_TV_20220902 = aggregate(os.path.join("assets", "trade-price-ir-vegas-20220902.csv"), 
                        {
                                "Value Date": {}, 
                                "Zero Rate Shock": {"aggregate": True}, 
                                "Expiry Bucket": {}, 
                                "Tenor Bucket": {"aggregate": True}, 
                        }, 
                        {
                                "TV": {}
                        })

from IPython.display import display
display(aspect_2_EB_TV_20220902)

In [ ]:
import os, sys; sys.path.insert(0, os.path.join(sys.path[0], '..'))
from scripts.data_processing_visualization import aggregate
aspect_2_TB_TV_20220902 = aggregate(os.path.join("assets", "trade-price-ir-vegas-20220902.csv"), 
                        {
                                "Value Date": {}, 
                                "Zero Rate Shock": {"aggregate": True}, 
                                "Expiry Bucket": {"aggregate": True}, 
                                "Tenor Bucket": {}, 
                        }, 
                        {
                                "TV": {}
                        })

from IPython.display import display
display(aspect_2_TB_TV_20220902)

#### Aggregate Vega for one day

Multiple vega values are averaged during aggregation. This approach may not be the best. 

As what you may have also noticed, when the ```Tenor Bucket``` is "18M", the Vega is always 0, indicating potential data loss when recording. To avoid that from affecting the aggregation result, remove all entries with the ```Tenor Bucket``` field equal to "18M". 

In [ ]:
import os, sys; sys.path.insert(0, os.path.join(sys.path[0], '..'))
from scripts.data_processing_visualization import aggregate
aspect_2_ZRS_Vega_20220902 = aggregate(os.path.join("assets", "trade-price-ir-vegas-20220902.csv"), 
                        {
                                "Value Date": {}, 
                                "Zero Rate Shock": {}, 
                                "Expiry Bucket": {"aggregate": True}, 
                                "Tenor Bucket": {
                                    "aggregate": True, 
                                    "values": ["1y", "2y", "3y", "4y", 
                                              "5y", "8y", "10y", "15y"]
                                    }, 
                        }, 
                        {
                                "Vega": {}
                        })

from IPython.display import display
display(aspect_2_ZRS_Vega_20220902)

In [ ]:
import os, sys; sys.path.insert(0, os.path.join(sys.path[0], '..'))
from scripts.data_processing_visualization import aggregate
aspect_2_EB_Vega_20220902 = aggregate(os.path.join("assets", "trade-price-ir-vegas-20220902.csv"), 
                        {
                                "Value Date": {}, 
                                "Zero Rate Shock": {"aggregate": True}, 
                                "Expiry Bucket": {}, 
                                "Tenor Bucket": {
                                    "aggregate": True, 
                                    "values": ["1y", "2y", "3y", "4y", 
                                              "5y", "8y", "10y", "15y"]
                                    }, 
                        }, 
                        {
                                "Vega": {}
                        })

from IPython.display import display
display(aspect_2_EB_Vega_20220902)

In [ ]:
import os, sys; sys.path.insert(0, os.path.join(sys.path[0], '..'))
from scripts.data_processing_visualization import aggregate
aspect_2_TB_Vega_20220902 = aggregate(os.path.join("assets", "trade-price-ir-vegas-20220902.csv"), 
                        {
                                "Value Date": {}, 
                                "Zero Rate Shock": {"aggregate": True}, 
                                "Expiry Bucket": {"aggregate": True}, 
                                "Tenor Bucket": {
                                    "values": ["1y", "2y", "3y", "4y", 
                                              "5y", "8y", "10y", "15y"]
                                    }, 
                        }, 
                        {
                                "Vega": {}
                        })

from IPython.display import display
display(aspect_2_TB_Vega_20220902)

Feel free to test the code yourself, and verify the following files generated: 

- ```trade-price-ir-vegas-20220902-id=expiryBucket_valueDate-d=tv.csv```

- ```trade-price-ir-vegas-20220902-id=expiryBucket_valueDate-d=vega.csv```

- ```trade-price-ir-vegas-20220902-id=tenorBucket_valueDate-d=tv.csv```

- ```trade-price-ir-vegas-20220902-id=tenorBucket_valueDate-d=vega.csv```

- ```trade-price-ir-vegas-20220902-id=valueDate_zeroRateShock-d=tv.csv```

- ```trade-price-ir-vegas-20220902-id=valueDate_zeroRateShock-d=vega.csv```

### Illustrate the change of ```TV``` and ```Vega```

- After clustering some independent variables, specify the target values of the remaining ones and observe how the dependent variable is further aggregated on top of them. 

- The function ```visualize_time_series``` implements this function. To show the change pattern of the dependent variable over time,  the "Value Date" column is always used as an independent variable without being clustered in ```visualize_time_series```.

- Generate a ```.csv``` file with all records in September 2022, from the original huge ```trade-price-ir-vegas.csv``` file. 

- The file ```trade-price-ir-vegas-timeranged-ed=30092022.csv``` should be available in the ```assets``` folder after running the following code. 

#### Obtain the Time Series dataframe

In [ ]:
import os, sys; sys.path.insert(0, os.path.join(sys.path[0], '..'))
from scripts.data_processing_visualization import time_ranged
aspect_2_TB_Vega_202209_time_ranged = time_ranged(os.path.join("assets", "trade-price-ir-vegas.csv"), 
            end_date="2022-09-30", save_file=True)

from IPython.display import display
display(aspect_2_TB_Vega_202209_time_ranged.head(), f"{aspect_2_TB_Vega_202209_time_ranged.shape[0]} rows in total")

In [ ]:
import os, sys; sys.path.insert(0, os.path.join(sys.path[0], '..'))
from scripts.data_processing_visualization import time_series
aspect_2_TB_Vega_202209_time_series = time_series(os.path.join("assets", "trade-price-ir-vegas-timeranged-ed=20220930.csv"), 
                        {
                                "Value Date": [None, None], 
                                "Zero Rate Shock": {}, 
                                "Expiry Bucket": {}, 
                                "Tenor Bucket": {
                                    "values": ["1y", "2y", "3y", "4y", 
                                              "5y", "8y", "10y", "15y"]
                                    }, 
                        }, 
                        {
                                "Vega": {}
                        }, save_file = True)

from IPython.display import display
display(aspect_2_TB_Vega_202209_time_series.head(), f"{aspect_2_TB_Vega_202209_time_series.shape[0]} rows in total")

## Regression Models

After running the above codes, we should obtain the following files: 

## Random Forest Models